In [2]:
import csv
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
import gensim
import operator
import csv

In [3]:
data= pd.read_csv(r'test.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'test.csv'

In [148]:
synopses=list(data["ملخص"])
names=list(data["اسم الفيلم"])
s=[]
n=[]
for i in range(len(synopses)):
    if synopses[i]!="ملخص":
        s.append(synopses[i])
        n.append(names[i])
synopses=s
names=n

In [149]:
names.index("حب وجنون")

263

In [150]:
%%time
def tagged_document(synopses):
    for i in range(len(synopses)):
        yield gensim.models.doc2vec.TaggedDocument(synopses[i].split(" "),[i])

data_for_training = list(tagged_document(synopses))


Wall time: 23.9 ms


In [151]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=40, min_count=2, epochs=30)


In [152]:
model.build_vocab(data_for_training)
model.train(data_for_training, total_examples=model.corpus_count, epochs=model.epochs)


In [153]:
def similarFilms(movieName):
    movieIndex=names.index(movieName)
    dic={}
    vec1=model.dv[movieIndex]
    for i in range(len(names)):
        if i!=movieIndex:
            vec2=model.dv[i]
            dic[i]=TS_SS(vec1,vec2)
    dic=dict(sorted(dic.items(), key=operator.itemgetter(1),reverse=False))
    dic=list(dic)

    answer=[]
    answer.append(names[dic[0]])
    answer.append(names[dic[1]])
    return answer

In [154]:
import math

def Cosine(vec1, vec2) :
    result = InnerProduct(vec1,vec2) / (VectorSize(vec1) * VectorSize(vec2))
    return result

def VectorSize(vec) :
    return math.sqrt(sum(math.pow(v,2) for v in vec))

def InnerProduct(vec1, vec2) :
    return sum(v1*v2 for v1,v2 in zip(vec1,vec2))

def Euclidean(vec1, vec2) :
    return math.sqrt(sum(math.pow((v1-v2),2) for v1,v2 in zip(vec1, vec2)))

def Theta(vec1, vec2) :
    return math.acos(Cosine(vec1,vec2)) + math.radians(10)

def Triangle(vec1, vec2) :
    theta = math.radians(Theta(vec1,vec2))
    return (VectorSize(vec1) * VectorSize(vec2) * math.sin(theta)) / 2

def Magnitude_Difference(vec1, vec2) :
    return abs(VectorSize(vec1) - VectorSize(vec2))

def Sector(vec1, vec2) :
    ED = Euclidean(vec1, vec2)
    MD = Magnitude_Difference(vec1, vec2)
    theta = Theta(vec1, vec2)
    return math.pi * math.pow((ED+MD),2) * theta/360

def TS_SS(vec1, vec2) :
    return Triangle(vec1, vec2) * Sector(vec1, vec2)

In [155]:
similarFilms("حب وجنون")

['خد الجميل', 'معبودة الجماهير']

In [158]:
similarFilms("ابن الشيطان")

['المتمرد', 'صقر العرب']